In [1]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [2]:
%pwd

'e:\\PROJECTS\\ML\\Text-Summarizer\\expriments'

In [3]:
%cd ..

e:\PROJECTS\ML\Text-Summarizer


c:\Users\reza\miniconda3\envs\textS\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%pwd

'e:\\PROJECTS\\ML\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataAcquisition:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


    
    def download_file(self):
        if not os.path.exists(self.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.source_URL,
                filename = self.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [6]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigHandler:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = self._load_config(config_filepath)
        self.params =  self._load_params(params_filepath)

        create_directories([self.config.artifacts_root])
    
    
    

    def _load_config(self, config_filepath):
        return read_yaml(config_filepath)  
    
    def _load_params(self, params_filepath):
        return read_yaml(params_filepath)
    
    def get_data_acquisition_config(self) -> DataAcquisition:
        config = self.config.data_acquisition

        create_directories([config.root_dir])

        data_acquisition_config = DataAcquisition(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_acquisition_config

In [ ]:
try:
    config = ConfigHandler()
    data_acquisition = config.get_data_acquisition_config()
    data_acquisition.download_file()
    data_acquisition.extract_zip_file()
except Exception as e:
    raise e